<div align = 'center'>
    <h1 id="section-0"><font face = 黑体> 企业税务风险事项判断及等级确定 </font>
<br>
<div>

<a id="ref11"></a>
### 实验步骤要求
<div class="alert alert-block alert-warning" style="margin-top: 20px">
在上一步实验中，我们通过编写计算企业风险指标总分并判断企业风险等级的函数，构建出了税务风险预警模型。在本步实验中，我们重点关注所选行业中的具有代表性的某个案例企业，希望能够对该企业的数据进行整合并计算相应风险评估指标，同时详细分析其近两年内存在税务风险事项，最后通过税务风险预警模型得到该企业的风险等级。
</div>

<a id = "ref12"></a>
### 数据说明
<div class="alert alert-block alert-success" style="margin-top: 20px">
案例企业的相关资料被存储于 data 文件夹下，文件名称为：“电商行业案例企业情况.docx”，包括企业基本情况、企业近2年财务报表、企业近2年纳税申报表等内容，您可以将该文件下载到本地进行查看。同时为便于对案例企业进行分析，我们已将文件中的数据进行了整合，详见文件“电商行业案例企业报表数据.xlsx”，关于该数据导入我们已经内置代码如下。
</div>

<a id = "ref13"></a>
### 步骤 1：阅读案例企业资料，将企业数据输入风险预警模型，获得各项指标的结果

In [1]:
import pandas as pd

df = pd.read_excel("data/电商行业案例企业报表数据.xlsx", sheet_name = '资产负债表')

sheet_names = ['利润表']
for sheet_name in sheet_names:
    df_ = pd.read_excel("data/电商行业案例企业报表数据.xlsx", sheet_name = sheet_name)
    df = pd.merge(df, df_, on = ['纳税人识别号', '纳税人名称', '统计截止日期'], how = 'left', suffixes=('', '_new'))

df_2021 = df[df['统计截止日期'] == '2021-12-31'].reset_index(drop = True)  ## 前期数据
df_2022 = df[df['统计截止日期'] == '2022-12-31'].reset_index(drop = True)  ## 基期数据
df_2023 = df[df['统计截止日期'] == '2023-12-31'].reset_index(drop = True)  ## 本期数据

print(df.shape)

# df

(3, 106)


<a id = "ref13"></a>
### 部分指标计算说明
<div class="alert alert-block alert-info" style="margin-top: 20px">
案例企业所包含数据相较于先前的财报数据更为丰富，因此在步骤 6 中未能实现的<strong>所选指标</strong>将在此计算，说明如下：

<li>1. “资源税税负变动系数”、“资源税同步增长系数”、“单位产成品原材料耗用率” 和 “投入产出评估分析指标”在案例企业中没有给出数据，不用计算；</li>
<li>2. “印花税税负变动系数”、“印花税同步增长系数”、“进项税额变动率与销项税额变动率配比” 和 “固定资产综合折旧率”只有部分行业案例企业给出数据，您需要对其计算可行性进行判断，并对可以计算的指标在下方编写函数完成计算过程。

&emsp;&emsp;关于具体代码的编写，下方预定义了一个名为 ExtendedIndicators 的子类，该子类继承了父类 FinancialIndicators 的所有属性和方法，并且可以添加自己的属性和方法，同时也可以对父类中的属性和方法进行覆写。子类中目前实现的 taxable_income_change_rate 函数已经对父类中关于“应纳税所得额变动率”指标的计算方法进行了覆写，您需要在该类中构造其他方法来计算上述说明 2 中的指标（如果可行）。这些指标的计算公式已在文件 “data/指标计算公式.txt” 中给出。
</div>

In [2]:
import numpy as np

import nbimporter
from warning_level import FinancialIndicators

class ExtendedIndicators(FinancialIndicators):
    def __init__(self, df_2023, df_2022, df_2021):

        # 调用父类的构造函数
        super().__init__(df_2023, df_2022, df_2021)
    
    ## 1. 应纳税所得额变动率
    '''
    应纳税所得额变动率 = (评估期累计应纳税所得额 - 基期累计应纳税所得额) / 基期累计应纳税所得额 * 100%
    财报中没有＂应纳税所得额＂，可用＂利润总额＂替代，但案例企业的企业所得税纳税申报表中有该项数据
    '''
    def taxable_income_change_rate(self):
        return '应纳税所得额变动率', self.calculate_change_rate(self.df_2023['应纳税所得额'], self.df_2022['应纳税所得额'])
    
    ##################              请在下方继续添加税务风险指标的计算函数              ##################
    
    ## 2. 印花税税负变动系数
    '''
    印花税税负变动系数 = 本期印花税负担率 / 上年同期印花税负担率
    印花税负担率 = (印花税应纳税额 / 印花税计税收入) * 100%
    '''
    # def stamp_tax_burden_change_coefficient(self):
    #     current_stamp_tax_burden_rate = self.df_2023['印花税应纳税额'] / self.df_2023['印花税计税收入'].replace(0, np.nan) * 100
    #     previous_stamp_tax_burden_rate = self.df_2022['印花税应纳税额'] / self.df_2022['印花税计税收入'].replace(0, np.nan) * 100
    #     return '印花税税负变动系数', current_stamp_tax_burden_rate / previous_stamp_tax_burden_rate

    ## 3. 印花税同步增长系数
    '''
    印花税同步增长系数 = 印花税应纳税额增长率 / 营业收入增长率
    '''
    # def stamp_tax_synchronous_growth_coefficient(self):
    #     current_taxable_amount_change_rate = self.calculate_change_rate(self.df_2023['印花税应纳税额'], self.df_2022['印花税应纳税额'])
    #     current_revenue_change_rate = self.calculate_change_rate(self.df_2023['营业收入'], self.df_2022['营业收入'])
    #     return '印花税同步增长系数', current_taxable_amount_change_rate / current_revenue_change_rate
    
    ## 4. 资源税税负变动系数
    '''
    资源税税负变动系数 = 本期资源税税收负担率 / 上年同期资源税税收负担率
    资源税税收负担率 = (资源税税额 / 销售收入)
    销售收入：利润表-营业收入
    '''
    # def resource_tax_burden_change_coefficient(self):
    #     current_resource_tax_burden_rate = self.df_2023['资源税税额'] / self.df_2023['营业收入'].replace(0, np.nan)
    #     previous_resource_tax_burden_rate = self.df_2022['资源税税额'] / self.df_2022['营业收入'].replace(0, np.nan)
    #     return '资源税税负变动系数', current_resource_tax_burden_rate / previous_resource_tax_burden_rate

    ## 5. 资源税同步增长系数
    '''
    资源税同步增长系数 = 资源税应纳税额增长率 / 营业收入(产品销售收入)增长率
    '''
    # def resource_tax_synchronous_growth_coefficient(self):
    #     current_taxable_amount_change_rate = self.calculate_change_rate(self.df_2023['资源税应纳税额'], self.df_2022['资源税应纳税额'])
    #     current_revenue_change_rate = self.calculate_change_rate(self.df_2023['营业收入'], self.df_2022['营业收入'])
    #     return '资源税同步增长系数', current_taxable_amount_change_rate / current_revenue_change_rate

    ## 6. 固定资产综合折旧率 **(只有一个案例企业有数据)
    '''
    固定资产综合折旧率 = 基期固定资产折旧总额 / 基期固定资产原值总额 * 100%
    '''
    # def comprehensive_depreciation_rate(self):
    #     pass

    ## 7. 进项税额变动率与销项税额变动率配比 **
    '''
    进项税额变动率与销项税额变动率配比 = 进项税额变动率 / 销项税额变动率
    '''
    # def input_output_tax_change_rate_ratio(self):
    #     input_tax_change_rate = self.calculate_change_rate(self.df_2023['进项税额'], self.df_2022['进项税额'])
    #     output_tax_change_rate = self.calculate_change_rate(self.df_2023['销项税额'], self.df_2022['销项税额'])
    #     return '进项税额变动率与销项税额变动率配比', input_tax_change_rate / output_tax_change_rate
    
    ##################              请在上方继续添加税务风险指标的计算函数              ##################

## 实例化类
indicators = ExtendedIndicators(df_2023, df_2022, df_2021)

## 获取类中的所有方法名
def get_class_methods(cls):
    attr_names = [attr_name for attr_name in dir(cls) if callable(getattr(cls, attr_name)) and not attr_name.startswith("__") and attr_name != 'calculate_change_rate']
    return attr_names

attr_names = get_class_methods(ExtendedIndicators)

## 创建一个字典, 用于存储各项税务风险指标的计算结果
dicts = {
    '纳税人识别号': df_2023['纳税人识别号'],
    '纳税人名称': df_2023['纳税人名称']
}

## 计算各项税务风险指标
for attr_name in attr_names:
    v_name, values = getattr(indicators, attr_name)()
    if v_name not in ['印花税税负变动系数', '印花税同步增长系数', '资源税税负变动系数', '资源税同步增长系数', '单位产成品原材料耗用率', '固定资产综合折旧率', '进项税额变动率与销项税额变动率配比', '投入产出评估分析指标']:  ## 需要排除的指标
        dicts[v_name] = values
    # print(v_name, values.iloc[0])

df = pd.DataFrame(dicts)

In [3]:
warning_interval = pd.read_csv("output/result_warning.csv", index_col = 0)

case_indicators = df.iloc[:,2:]
case_indicators.index = ['案例企业指标值']

if warning_interval.shape[1] != case_indicators.shape[1]:
    raise ValueError("案例企业计算指标和行业指标对不上!")

indicator_info = pd.concat([warning_interval.iloc[3:, :], case_indicators], axis = 0)
indicator_info

,研发费用变动率,预收款变动率,资产负债率,资产利润率配比,营业成本变动率,营业成本变动率与营业利润变动率配比,营业费用变动率,营业利润变动率,营业利润税收负担率,营业收入变动率,...,其他应收款变动率,应付账款变动率,应收账款变动率,净资产收益率,销售利润率配比,应纳税额与工业增加值弹性系数,应纳税所得额变动率,总资产周转率,总资产周转率配比,增值税税收负担率
预警值下限,-53.877237,-27.351784,23.029251,0.700000,-20.837415,0.700000,-11.463036,-149.414031,5.746523,-26.661024,...,-27.452619,-34.776103,-35.686385,-17.214780,0.700000,-0.058894,-173.658309,60.115953,0.700000,1.936173
预警值上限,23.063242,61.390963,56.088061,1.300000,18.882373,1.300000,21.207042,50.143673,17.795532,22.906918,...,61.713245,30.698037,26.097758,14.891971,1.300000,0.100663,58.947875,128.007894,1.300000,5.569832
案例企业指标值,68.566261,476.945075,52.411652,-1.901133,60.219509,-1.963144,5.322565,-30.675040,4.030109,26.282599,...,-69.069210,141.469387,13.773232,NaN,-12.366759,-0.750587,NaN,24.141359,3.949384,21.753663


<a id = "ref15"></a>
### 步骤 2：判断案例企业的税务风险事项

In [4]:
'''
详细阅读文件中关于案例企业近两年的情况介绍，对案例企业可能存在的税务风险事项做出判断，以下为示例：

税务风险事项 1: 
    风险点: 多扣费用少缴企业所得税风险
    风险描述: 企业202X年取得发票金额199.2万元, 而确认管理费用222.6万元, 部分费用存在无发票进行税前扣除的情况, 存在少缴企业所得税风险
税务风险事项 2: 
    风险点: ......
    风险描述: ......
......

请在下方填写您对案例企业可能存在的税务风险事项的判断：








'''
print('\t')

<a id = "ref16"></a>
### 步骤 3：判断案例企业的风险等级

In [5]:
import nbimporter
from model import risk_level_calculation

import json

## 载入对所选指标的分组信息（配比或非配比）
with open('data/配比指标划分.json', 'r') as file:
    ratio_ratio_group_infos = json.load(file)

'''
ratio_ratio_group_infos = {
    'non_ratio_ratio_indicator': ['indicator1', 'indicator2', 'indicator3', ...],  ## 非配比指标
    'ratio_ratio_indicator': ['indicator4', 'indicator5', 'indicator6', ...]  ## 配比指标
}
'''
# print(ratio_ratio_group_infos)

weights = pd.read_csv("output/weights.csv", index_col = 0)

## 这里为测试方便，只选择了部分指标，实际中下面一行代码需注释掉
df = df[[
    '纳税人识别号',
    '纳税人名称',
    '营业收入变动率',
    '营业成本变动率',
    '营业费用变动率',
    '销售费用变动率',
    '管理费用变动率',
    '财务费用变动率',
    '研发费用变动率',
    '成本费用利润率',
    '营业利润变动率',
    '营业外收入变动率',
    '营业收入变动率与营业利润变动率配比',
    '营业收入变动率与营业成本变动率配比',
    '营业收入变动率与营业费用变动率配比',
    '营业成本变动率与营业利润变动率配比',
    '增值税税收负担率',
    '应纳税额与工业增加值弹性系数',
    '工业增加值税负差异率'
]]

scores_unweighted, score_sum, risk_level = risk_level_calculation(df, warning_interval = warning_interval, weights = weights, ratio_ratio_group_infos = ratio_ratio_group_infos)

非配比指标:  营业收入变动率   -26.66102360940085   22.906917953924733
非配比指标:  营业成本变动率   -20.837415379193004   18.88237290077088
非配比指标:  营业费用变动率   -11.4630355035277   21.20704194945469
非配比指标:  销售费用变动率   -13.552241769036474   19.85824858823209
非配比指标:  管理费用变动率   -10.693134048016107   12.6550633924579
非配比指标:  财务费用变动率   -83.38565493946244   51.31382360217636
非配比指标:  研发费用变动率   -53.87723723644554   23.06324150145436
非配比指标:  成本费用利润率   -4.249123646883749   4.684789187274348
非配比指标:  营业利润变动率   -149.41403082536678   50.14367252194993
非配比指标:  营业外收入变动率   -95.37221556678826   5.9843208623186825
配比指标:  营业收入变动率与营业利润变动率配比   0.7   1.3
配比指标:  营业收入变动率与营业成本变动率配比   0.7   1.3
配比指标:  营业收入变动率与营业费用变动率配比   0.7   1.3
配比指标:  营业成本变动率与营业利润变动率配比   0.7   1.3
非配比指标:  增值税税收负担率   1.936172515265501   5.569831913260073
非配比指标:  应纳税额与工业增加值弹性系数   -0.0588944160872073   0.1006634996927278
非配比指标:  工业增加值税负差异率   -0.3608952494560207   2.3608952494560205


In [6]:
scores_unweighted

,纳税人识别号,纳税人名称,营业收入变动率,营业成本变动率,营业费用变动率,销售费用变动率,管理费用变动率,财务费用变动率,研发费用变动率,成本费用利润率,营业利润变动率,营业外收入变动率,营业收入变动率与营业利润变动率配比,营业收入变动率与营业成本变动率配比,营业收入变动率与营业费用变动率配比,营业成本变动率与营业利润变动率配比,增值税税收负担率,应纳税额与工业增加值弹性系数,工业增加值税负差异率
0,暂无,甲电商零售有限公司,25.0,0,100,100,20.0,100,73.0,0,100,92.0,93.0,99.0,82.0,87.0,77.0,0,100


In [7]:
score_sum

,纳税人识别号,纳税人名称,风险总分
0,暂无,甲电商零售有限公司,62.2981


In [8]:
risk_level

,纳税人识别号,纳税人名称,风险等级
0,暂无,甲电商零售有限公司,中风险
